<a href="https://colab.research.google.com/github/zanuarts/dicoding-BPML/blob/master/dicoding_14_Deploy_NLP_TFJS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['KAGGLE_USERNAME']= 'zanuarer'
os.environ['KAGGLE_KEY'] = '27695b9355b4fb42fec2d5a412728bec'

In [2]:
!kaggle datasets download -d marklvl/sentiment-labelled-sentences-data-set

  0% 0.00/326k [00:00<?, ?B/s]
100% 326k/326k [00:00<00:00, 49.0MB/s]


In [3]:
!unzip -q sentiment-labelled-sentences-data-set.zip -d .

In [4]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

df = pd.read_csv('/content/sentiment labelled sentences/yelp_labelled.txt', names=['sentence', 'label'], sep='\t')
df.shape

(1000, 2)

In [5]:
df.head

<bound method NDFrame.head of                                               sentence  label
0                             Wow... Loved this place.      1
1                                   Crust is not good.      0
2            Not tasty and the texture was just nasty.      0
3    Stopped by during the late May bank holiday of...      1
4    The selection on the menu was great and so wer...      1
..                                                 ...    ...
995  I think food should have flavor and texture an...      0
996                           Appetite instantly gone.      0
997  Overall I was not impressed and would not go b...      0
998  The whole experience was underwhelming, and I ...      0
999  Then, as if I hadn't wasted enough of my life ...      0

[1000 rows x 2 columns]>

In [6]:
df['sentence'] = df['sentence'].str.lower()

In [7]:
# remove stopwords

from nltk.corpus import stopwords #comment jika Error dan gunakan 2 sintaks dibawah
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
stop = set(stopwords.words('english'))
df['sentence'] = df['sentence'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop)]))
df.head()

,sentence,label
0,wow... loved place.,1
1,crust good.,0
2,tasty texture nasty.,0
3,stopped late may bank holiday rick steve recom...,1
4,selection menu great prices.,1


In [9]:
vocab_size = 2000
oov_tok = "<OOV>"
filt = '!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ' #remove symbols

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok, filters = filt)
tokenizer.fit_on_texts(df['sentence'].values)

word2index = tokenizer.word_index
print(len(word2index))

1998


In [10]:
import json

with open('word2index.json', 'w') as fp:
    json.dump(word2index, fp)

In [11]:
max_length =  max(len(values.split()) for i, values in enumerate(df['sentence']))
max_length

18

In [12]:
trunc_type='post'

all_seq = tokenizer.texts_to_sequences(df['sentence'].values)
all_padded = pad_sequences(all_seq, maxlen = max_length, padding = trunc_type)
all_padded.shape

(1000, 18)

In [13]:
# split train and test sets
from sklearn.model_selection import train_test_split

X = all_padded
y = pd.get_dummies(df['label'].values)
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

kalimat = df['sentence'].values
y = df['label'].values

kalimat_latih, kalimat_test, y_latih, y_test = train_test_split(kalimat, y, 
                                                               test_size=0.2, random_state=1000)

(800, 18) (800,)
(200, 18) (200,)


In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim= vocab_size, output_dim=16, input_length= max_length),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 18, 16)            32000     
_________________________________________________________________
lstm (LSTM)                  (None, 64)                20736     
_________________________________________________________________
dense (Dense)                (None, 24)                1560      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 54,321
Trainable params: 54,321
Non-trainable params: 0
_________________________________________________________________


In [15]:
num_epochs = 30
history = model.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_test, y_test))

Epoch 1/30
25/25 [==============================] - 1s 33ms/step - loss: 0.6940 - accuracy: 0.4913 - val_loss: 0.6928 - val_accuracy: 0.5200
Epoch 2/30
25/25 [==============================] - 0s 11ms/step - loss: 0.6933 - accuracy: 0.4800 - val_loss: 0.6933 - val_accuracy: 0.4800
Epoch 3/30
25/25 [==============================] - 0s 11ms/step - loss: 0.6904 - accuracy: 0.5100 - val_loss: 0.6950 - val_accuracy: 0.4850
Epoch 4/30
25/25 [==============================] - 0s 10ms/step - loss: 0.4777 - accuracy: 0.7950 - val_loss: 1.1296 - val_accuracy: 0.5450
Epoch 5/30
25/25 [==============================] - 0s 11ms/step - loss: 0.1493 - accuracy: 0.9438 - val_loss: 1.1889 - val_accuracy: 0.5300
Epoch 6/30
25/25 [==============================] - 0s 11ms/step - loss: 0.0806 - accuracy: 0.9812 - val_loss: 2.1483 - val_accuracy: 0.5400
Epoch 7/30
25/25 [==============================] - 0s 11ms/step - loss: 0.0530 - accuracy: 0.9875 - val_loss: 2.0480 - val_accuracy: 0.5400
Epoch 8/30
25

In [16]:
!pip install tensorflowjs

     |████████████████████████████████| 71kB 4.3MB/s 
     |████████████████████████████████| 92kB 8.1MB/s 
     |████████████████████████████████| 137.3MB 74kB/s 
     |████████████████████████████████| 256kB 43.4MB/s 
     |████████████████████████████████| 921kB 44.6MB/s 
  Created wheel for PyInquirer: filename=PyInquirer-1.0.3-cp36-none-any.whl size=32852 sha256=480081bae6aa4431b2d53c834d1cc4e70beea92fbfba569175ce13e6ea21cd09
  Stored in directory: /root/.cache/pip/wheels/52/6c/b1/3e4b0e8daf42a92883c7641c0ea8ffb62e0490ebed2faa55ad
Successfully built PyInquirer
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Found existing installation: Pygments 2.1.3
    Uninstalling Pygments-2.1.3:
      Successfully uninstalled Pygments-2.1.3
  Found existing installation: tensorflow-hub 0.

In [17]:
saved_model_path = '/content/mymodel/'
tf.saved_model.save(model, saved_model_path)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /content/mymodel/assets


In [18]:
!tensorflowjs_converter \
  --input_format=tf_saved_model \
  /content/mymodel/ \
  /content/modeltfjs

2020-09-09 06:02:49.594532: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2020-09-09 06:02:49.598733: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2000165000 Hz
2020-09-09 06:02:49.598965: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1fd6d80 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-09-09 06:02:49.599003: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-09-09 06:02:51.068139: I tensorflow/core/grappler/devices.cc:78] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2020-09-09 06:

In [23]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer (num_words = vocab_size, oov_token = "<OOV>")
tokenizer.fit_on_texts(df['sentence'].values)

word2index = tokenizer.word_index

In [24]:
import json
with open('word2index.json', 'w') as fp:
    json.dump(word2index, fp)